In [ ]:
import sys
sys.path.append('src')

***
## Module Imports

In [ ]:
import nltk
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from nltk.metrics.distance import jaccard_distance
from collections.abc import Iterable
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [ ]:
from data_utils import load_data
from dimension.lexical import *
from dimension.syntactical import *

***
## Data

In [ ]:
train_data, test_data = load_data('data/')
print(
    f"train_data samples: {len(train_data)}, test_data samples: {len(test_data)}"
)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

***
## Similarity functions

In [ ]:
def jaccard_similarity(s1, s2):
    assert isinstance(s1, Iterable), f"s1 must be an iterable, not {type(s1)}"
    assert isinstance(s2, Iterable), f"s2 must be an iterable, not {type(s2)}"
    return 1 - jaccard_distance(set(s1), set(s2))

In [ ]:
def overlap_similarity(s1, s2):
    assert isinstance(s1, Iterable), f"s1 must be an iterable, not {type(s1)}"
    assert isinstance(s2, Iterable), f"s2 must be an iterable, not {type(s2)}"
    s1 = set(s1)
    s2 = set(s2)
    intersection = s1.intersection(s2)
    return len(intersection) / min(len(s1), len(s2))

In [ ]:
def cosine_similarity(s1, s2):
    assert isinstance(s1, Iterable), f"s1 must be an iterable, not {type(s1)}"
    assert isinstance(s2, Iterable), f"s2 must be an iterable, not {type(s2)}"
    s1 = set(s1)
    s2 = set(s2)
    intersection = s1.intersection(s2)
    return len(intersection) / ((len(s1) * len(s2))**2)

In [ ]:
def dice_similarity(s1, s2):
    assert isinstance(s1, Iterable), f"s1 must be an iterable, not {type(s1)}"
    assert isinstance(s2, Iterable), f"s2 must be an iterable, not {type(s2)}"
    s1 = set(s1)
    s2 = set(s2)
    intersection = s1.intersection(s2)
    return 2 * len(intersection) / (len(s1) + len(s2))

***
## Feature loading

### feature vector builder for dataframe of sentence pairs

Declaration of the function responsible for the iteration over the dataframe containing the sentence pairs (other columns shall be unused). Requires the sentences columns' to be named `"S1"` and `"S2"`.

Returns a numpy array of shape `(n_sentence_pairs, n_features)`

In [ ]:
def get_features(df: pd.DataFrame):
    assert "S1" in df.columns, "S1 not in dataframe"
    assert "S2" in df.columns, "S2 not in dataframe"

    features = [None] * len(df)   #preallocated for memory efficiency

    for index, row in df.iterrows():
        sentence1, sentence2 = row['S1'], row['S2']

        # Get all words
        tokenized_1, tokenized_2 = get_tokenized_sentences(
            sentence1, sentence2, return_unique_words=False)
        tokenized_lc_1, tokenized_lc_2 = get_tokenized_sentences_lowercase(
            tokenized_1, tokenized_2, return_unique_words=False)

        # Get words without stopwords
        no_stopwords_1, no_stopwords_2 = filter_stopwords(
            tokenized_1, tokenized_2, return_unique_words=False)
        no_stopwords_lc_1, no_stopwords_lc_2 = filter_stopwords(
            tokenized_lc_1, tokenized_lc_2, return_unique_words=False)

        # Lemmas
        lemmatized_1, lemmatized_2 = get_lemmas(tokenized_1,
                                                tokenized_2,
                                                return_unique_words=False)
        lemmatized_lc_1, lemmatized_lc_2 = get_lemmas(
            tokenized_lc_1, tokenized_lc_2, return_unique_words=False)

        # Name entities
        sentence_ne_1, sentence_ne_2 = get_named_entities(
            tokenized_1, tokenized_2)
        
        #lemmas cleaned from stopwords
        stopwords_and_lemmas1, stopwords_and_lemmas2 = get_lemmas(
            no_stopwords_1, no_stopwords_2, return_unique_words=False)

        stopwords_and_lemmas_lc_1, stopwords_and_lemmas_lc_2 = get_lemmas(
            no_stopwords_lc_1, no_stopwords_lc_2, return_unique_words=False)
        
        # Name entities without stopwords in lowercase
        ne_no_stopwords_1, ne_no_stopwords_2 = filter_stopwords(
            sentence_ne_1, sentence_ne_2, return_unique_words=False, filter_and_return_in_lowercase=True)
        
        # Name entities without stopwords in lowercase and lemmas
        ne_no_stopwords_lemmas_1, ne_no_stopwords_lemmas_2 = get_lemmas( ne_no_stopwords_1, ne_no_stopwords_2, 
                                                                        return_unique_words=False)

        # Bigrams
        bigrams_1, bigrams_2 = get_ngrams(no_stopwords_1, no_stopwords_2, n=2)
        trigrams_1, trigrams_2 = get_ngrams(no_stopwords_1, no_stopwords_2, n=3)
        
        # Bigrams trigrams with sentence tokenizer 
        bigrams_sent_1, bigrams_sent_2 = get_ngrams_with_sent_tokenize(sentence1, sentence2, n=2)
        trigrams_sent_1, trigrams_sent_2 = get_ngrams_with_sent_tokenize(sentence1, sentence2, n=3)
        
        # Lesk
        lesk_1, lesk_2 = get_lesk_sentences(tokenized_1, tokenized_2)

        # Stemmer
        stemmed_1, stemmed_2 = get_stemmed_sentences(sentence1, sentence2)
        
        # ALL Features
        features[index] = [
            jaccard_similarity(tokenized_1, tokenized_2),
            jaccard_similarity(tokenized_lc_1, tokenized_lc_2),
            jaccard_similarity(no_stopwords_1, no_stopwords_2),
            jaccard_similarity(no_stopwords_lc_1, no_stopwords_lc_2),
            jaccard_similarity(lemmatized_1, lemmatized_2),
            jaccard_similarity(lemmatized_lc_1, lemmatized_lc_2),
            jaccard_similarity(sentence_ne_1, sentence_ne_2),
            jaccard_similarity(stopwords_and_lemmas1, stopwords_and_lemmas2),
            jaccard_similarity(stopwords_and_lemmas_lc_1, stopwords_and_lemmas_lc_2),
            jaccard_similarity(bigrams_1, bigrams_2),
            jaccard_similarity(trigrams_1, trigrams_2),
            jaccard_similarity(bigrams_sent_1, bigrams_sent_2),
            jaccard_similarity(trigrams_sent_1, trigrams_sent_2),
            jaccard_similarity(lesk_1, lesk_2),
            jaccard_similarity(stemmed_1, stemmed_2),
            
            dice_similarity(tokenized_1, tokenized_2),
            dice_similarity(tokenized_lc_1, tokenized_lc_2),
            dice_similarity(no_stopwords_1, no_stopwords_2),
            dice_similarity(no_stopwords_lc_1, no_stopwords_lc_2),
            dice_similarity(lemmatized_1, lemmatized_2),
            dice_similarity(lemmatized_lc_1, lemmatized_lc_2),
            dice_similarity(sentence_ne_1, sentence_ne_2),
            dice_similarity(stopwords_and_lemmas1, stopwords_and_lemmas2),
            dice_similarity(stopwords_and_lemmas_lc_1, stopwords_and_lemmas_lc_2),
            dice_similarity(bigrams_1, bigrams_2),
            dice_similarity(trigrams_1, trigrams_2),
            dice_similarity(bigrams_sent_1, bigrams_sent_2),
            dice_similarity(trigrams_sent_1, trigrams_sent_2),
            dice_similarity(lesk_1, lesk_2),
            dice_similarity(stemmed_1, stemmed_2)
        ]
        # BEST Features selection 
        """features[index] = [
        jaccard_similarity(tokenized_1, tokenized_2),
        jaccard_similarity(no_stopwords_1, no_stopwords_2),
        jaccard_similarity(lemmatized_1, lemmatized_2),
        jaccard_similarity(bigrams_1, bigrams_2),
        jaccard_similarity(trigrams_1, trigrams_2),
        dice_similarity(tokenized_1, tokenized_2),
        dice_similarity(no_stopwords_1, no_stopwords_2),
        dice_similarity(no_stopwords_lc_1, no_stopwords_lc_2),
        dice_similarity(sentence_ne_1, sentence_ne_2),
        dice_similarity(bigrams_1, bigrams_2),
        dice_similarity(trigrams_1, trigrams_2),
        dice_similarity(stopwords_and_lemmas_lc_1, stopwords_and_lemmas_lc_2),
        dice_similarity(lesk_1, lesk_2)
        ]"""
    return np.array(features)

In [ ]:
# TEST cell don't delete it =D

first = "My Bonnie White lies over the ocean, in Picadilli Circus at 3:00pm."
second = "My Bonnie lied over the sea! Over the sea..."

tokenized_1, tokenized_2 = get_tokenized_sentences(first,
                                                   second,
                                                   return_unique_words=False)
no_stopwords_1, no_stopwords_2 = filter_stopwords(tokenized_1, tokenized_2, return_unique_words=False)
sentence_ne_1, sentence_ne_2 = get_named_entities(no_stopwords_1, no_stopwords_2)

print(tokenized_1)
print(tokenized_2)
print(sentence_ne_1)
print(sentence_ne_2)
#TEST cell

### Train features extraction

Using the function declared above, the features are extracted from the `train_data` dataframe. Also the Gold Standard is extracted from its column in the dataframe. The shapes for both numpy vectors are displayed. 

In [ ]:
train_features = get_features(train_data)
train_gs = train_data['Gs'].to_numpy()
print(f"train_features.shape: {train_features.shape}")
print(f"train_gs.shape: {train_gs.shape}")

### Test features extraction

In [ ]:
test_features = get_features(test_data)
test_gs = test_data['Gs'].to_numpy()
print(f"train_features.shape: {test_features.shape}")
print(f"train_gs.shape: {test_gs.shape}")

### Feature scaling

features are scaled using sklearns StandardScaler, where the mean is substracted for each feature and it's divided by the variance of the feature to obtain a unified feature space with zero mean and unit variance.

In [ ]:
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)

In [ ]:
test_features_scaled = scaler.transform(test_features)

### Predefined Fold for GridSearch

In [ ]:
from sklearn.model_selection import PredefinedSplit
all_data = np.concatenate([train_features_scaled, test_features_scaled])
all_labels = np.concatenate([train_gs, test_gs])
test_fold = np.array([-1]*train_features_scaled.shape[0] + [0]*test_features_scaled.shape[0])
print(all_data.shape, test_fold.shape)
ps = PredefinedSplit(test_fold)

In [ ]:
# testing if the predefined split is correct
train_idx, test_idx = list(ps.split())[0]
print(train_idx.shape, test_idx.shape)
print(train_features_scaled.shape, test_features_scaled.shape)

assert np.all(all_data[train_idx] == train_features_scaled), "incorrect split"
assert np.all(all_data[test_idx] == test_features_scaled), "incorrect split"

### SVR

In [ ]:
pearson_scorer = make_scorer(lambda y, y_hat: pearsonr(y, y_hat)[0])

gammas = np.logspace(-6, -1, 6)
Cs = np.array([0.5, 1, 2, 4, 8, 10, 15, 20, 50, 100, 200, 375, 500, 1000])
epsilons = np.linspace(0.1, 1, 10)
svm_param = dict(gamma=gammas, C=Cs, epsilon=epsilons)

svr = SVR(kernel='rbf', tol=1)
gssvr = GridSearchCV(svr,
                     svm_param,
                     cv=ps,
                     scoring=pearson_scorer,
                     n_jobs=-1,
                     verbose=1)
gssvr = gssvr.fit(all_data, all_labels)

In [ ]:
svr_best_params = gssvr.best_params_
best_svr = SVR(kernel='rbf', tol=1, **svr_best_params)
best_svr.fit(train_features_scaled, train_gs)

In [ ]:
train_svm_predictions = best_svr.predict(train_features_scaled)

In [ ]:
%%time
test_svm_predictions = best_svr.predict(test_features_scaled)

In [ ]:
train_svm_correlation = pearsonr(train_svm_predictions, train_gs)[0]
test_svm_correlation = pearsonr(test_svm_predictions, test_gs)[0]

In [ ]:
print('Train pearsonr:', train_svm_correlation)
print("Test pearsonr: ", test_svm_correlation)
print('The best value of gamma:', gssvr.best_estimator_.gamma)
print('The best value of C:', gssvr.best_estimator_.C)
print('The best value of epsilon:', gssvr.best_estimator_.epsilon)

### MLP Regressor

In [ ]:
from sklearn.neural_network import MLPRegressor

alphas = np.logspace(-6, -1, 6)
hidden_layer_sizes = [(i,) for i in range(5, 305, 10)]
mlp_param = dict(alpha=alphas, hidden_layer_sizes=hidden_layer_sizes)

mlpr = MLPRegressor(max_iter=1000, random_state=1)
mgsmlp = GridSearchCV(mlpr,
                      mlp_param,
                      cv=ps,
                      scoring=pearson_scorer,
                      n_jobs=-1,
                      verbose=1)
mgsmlp = mgsmlp.fit(all_data, all_labels)

In [ ]:
mlp_best_params = mgsmlp.best_params_
best_mlp = MLPRegressor(max_iter=1000, random_state=1, **mlp_best_params)
best_mlp.fit(train_features_scaled, train_gs)

In [ ]:
train_mlp_predictions = best_mlp.predict(train_features_scaled)

In [ ]:
%%time
test_mlp_predictions = best_mlp.predict(test_features_scaled)

In [ ]:
train_mlp_correlation = pearsonr(train_mlp_predictions, train_gs)[0]
test_mlp_correlation = pearsonr(test_mlp_predictions, test_gs)[0]
print('Train pearsonr:', train_mlp_correlation)
print("Test pearsonr: ", test_mlp_correlation)
print('The best value of alpha:', mgsmlp.best_estimator_.alpha)
print('The best value of hidden_layer_sizes:', mgsmlp.best_estimator_.hidden_layer_sizes)

### NN Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

weights = ['uniform', 'distance']
nn = list(range(10, 101, 10))
algorithms = ['ball_tree', 'kd_tree', 'brute']
leaf_sizes = list(range(1, 51, 10))
pvalues = list(range(1, 6))

nn_params = dict(n_neighbors=nn, weights=weights, p=pvalues, leaf_size=leaf_sizes, algorithm=algorithms)

nnr = KNeighborsRegressor()
nnrmlp = GridSearchCV(nnr,
                      nn_params,
                      cv=ps,
                      scoring=pearson_scorer,
                      n_jobs=-1,
                      verbose=1)

nnrmlp = nnrmlp.fit(all_data, all_labels)

In [ ]:
nnr_best_params = nnrmlp.best_params_
best_nnr = KNeighborsRegressor(**nnr_best_params)
best_nnr.fit(train_features_scaled, train_gs)

In [ ]:
train_nnr_predictions = best_nnr.predict(train_features_scaled)

In [ ]:
%%time
test_nnr_predictions = best_nnr.predict(test_features_scaled)

In [ ]:
train_nnr_correlation = pearsonr(train_nnr_predictions, train_gs)[0]
test_nnr_correlation = pearsonr(test_nnr_predictions, test_gs)[0]
print('Train pearsonr: ', train_nnr_correlation)
print("Test pearsonr: ", test_nnr_correlation)
print('The best value of n_neighbors:', nnrmlp.best_estimator_.n_neighbors)
print('The best value of weights:', nnrmlp.best_estimator_.weights)
print('The best value of p:', nnrmlp.best_estimator_.p)
print('The best value of leaf_size:', nnrmlp.best_estimator_.leaf_size)
print('The best value of algorithm:', nnrmlp.best_estimator_.algorithm)

### Bottleneck Layer MLP

In [ ]:
from itertools import product
bottleneck_length = range(3, test_features.shape[-1])
hidden_layer_length = range(4, 100, 20)
alphas_lin = np.linspace(0, 1, 6)
hidden_layer_sizes = list(product(bottleneck_length, hidden_layer_length))

param = dict(hidden_layer_sizes=hidden_layer_sizes, alpha=alphas_lin)

bmlp = MLPRegressor(max_iter=500, random_state=1)
gsbmlp = GridSearchCV(bmlp,
                          param,
                          cv=ps,
                          scoring=pearson_scorer,
                          n_jobs=-1,
                          verbose=1)
gsbmlp = gsbmlp.fit(all_data, all_labels)

In [ ]:
bt_mlp_best_params = gsbmlp.best_params_
best_bt_mlp = MLPRegressor(max_iter=500, random_state=1, **bt_mlp_best_params)
best_bt_mlp.fit(train_features_scaled, train_gs)

In [ ]:
train_bt_mlp_predictions = best_bt_mlp.predict(train_features_scaled)

In [ ]:
%%time
test_bt_mlp_predictions = best_bt_mlp.predict(test_features_scaled)

In [ ]:
train_bt_mlp_correlation = pearsonr(train_bt_mlp_predictions, train_gs)[0]
test_bt_mlp_correlation = pearsonr(test_bt_mlp_predictions, test_gs)[0]
print('Train pearsonr: ', train_bt_mlp_correlation)
print('Test pearsonr: ', test_bt_mlp_correlation)
print('The best value of alpha:', gsbmlp.best_estimator_.alpha)
print('The best value of hidden_layer_sizes:', gsbmlp.best_estimator_.hidden_layer_sizes)

***
## Results

In [ ]:
correlations = {
    "SVR": {"train": train_svm_correlation,"test": test_svm_correlation},
    "MLP": {"train": train_mlp_correlation,"test": test_mlp_correlation},
    "NNR": {"train": train_nnr_correlation,"test": test_nnr_correlation},
    "BtBLP": {"train": train_bt_mlp_correlation,"test": test_bt_mlp_correlation},
}
print(pd.DataFrame.from_dict(correlations, orient="index"))